In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import artpop
import astropy.units as u
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.visualization import make_lupton_rgb

In [ ]:
def perturb_mags(model_filts, input_colors, input_mags, input_color_err, input_mag_err):
    """
    Make model CMD more realistic by 1. matching the limiting magnitude and 2. perturbing ArtPop stellar magnitudes by the errors on the input CMD.
    
    PARAMETERS:
        model_filts (str, arr-like): list of filters used in CMD -- [bluer filter, redder filter], e.g. ["ACS_WFC_F606W", "ACS_WFC_F814W"]
        input_colors (arr-like): x-axis of data-based CMD, e.g. F606W-F814W
        input_mags (arr-like): y-axis of data-based CMD, e.g. F814W
        input_color_errs (arr-like): error on x-axis of data-based CMD
        input_mag_errs (arr-like): error y-axis of data-based CMD
    
    
    RETURNS: 
        CMD x-axis (perturbed model colors, array), CMD y-axis (perturbed model magnitudes, array) 
    """
    
    pop = ssp.star_mags(model_filts[1])
    pop_color = ssp.star_mags(model_filts[0]) - ssp.star_mags(model_filts[1])

    ## pop is the joint CMD of old and young model populations, pop_color is the color of joint CMD
    x_err_mod = []#append the nearest errors here
    y_err_mod = []
    new_pop = []
    new_popcolor = []
    mult = (-1., 1.) #so applying the error becomes mult[np.random.randint(2, size = 1)]

    use_814 = np.array(F814W_vega[inBlob_corr]) #+ 0.45 can apply later since ArtPop also reads out as Vega mag
    use_606 = np.array(F606W_vega[inBlob_corr]) #+ 0.02


    for i in range(len(pop)):
        y_err_new = y_err[np.argmin(abs(use_814 - pop[i]))]
        y_err_mod.append(y_err_new)
        new_pop.append(pop[i] + gauss(0, y_err_new**2))

        x_err_new = x_err[np.argmin(abs(use_814 - pop[i]))]
        x_err_mod.append(x_err_new)
        new_popcolor.append(pop_color[i] + gauss(0, x_err_new**2))
        
    return new_popcolor, new_pop

In [ ]:
def plot_results(img_filts, reff, n, theta, ellip, distance, pixel_scale, CMD_filts = img_filts[-2:], zpts = None, psfs = None, xy_dim = 1001, Q = 0.5, stretch = 0.06, save = True)
    stars = ssp.abs_mag_table

    xy = artpop.sersic_xy(
        num_stars = len(stars),     
        r_eff = reff*u.kpc,         # effective radius (kpc)
        n = n,             # Sersic index
        theta = theta*u.deg,          # position angle (deg)
        ellip = ellip,         # ellipticity
        distance = distance*u.Mpc,       # distance to system (Mpc)
        xy_dim = xy_dim, # xy dimensions of image
        pixel_scale = pixel_scale
    )
    
    distance_mod = 5*np.log10(distance*u.Mpc.to(u.pc)/10)
    
    mags_ = {'ACS_WFC_F475W': stars['WFC_ACS_F475W'] + distance_mod, 'ACS_WFC_F606W': stars['WFC_ACS_F606W'] + distance_mod, 
         'ACS_WFC_F814W': stars['WFC_ACS_F814W'] + distance_mod} #might need to convert names/between abs mag and app mag; will need to see as we work on it
    
    ssp = artpop.source.Source(xy, mags = mags_, xy_dim = 1001, pixel_scale = 0.05)
    
    img = {}
    filt = img_filts
    zpt_ = zpts
    psf_ = psfs
    for i in range(len(filt)):
        img[filt[i]] = imager.observe(ssp, bandpass = img_filt[i], psf = psf_[i], zpt = zpt_[i]).image
    
    if (len(img_filts) > 3) or (len(img_filts) < 2):
        raise Exception("Must have two or three filters to generate mock image.")
    
    if len(img_filts) == 2:
        b = img[img_filts[0]]
        g = (img[img_filts[0]] + img[img_filts[1]])/2
        r = img[img_filts[1]]
        
    else:
        b = img[img_filts[0]]
        g = img[img_filts[1]]
        r = img[img_filts[2]]

    rgb = make_lupton_rgb(r, g, b, Q=Q, stretch=stretch)